In [ ]:
#gils code for plots. Example use below. This plots a scaling too. 
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
import os
from matplotlib.colors import LinearSegmentedColormap, Normalize
from matplotlib.colorbar import ColorbarBase

def plot_counties_from_csv(shapefile_gdf, csv_file_path, output_folder, title_format="Plot for {}"):
    df = pd.read_csv(csv_file_path)
    df.rename(columns={'state_fips': 'STATEFP', 'subregion2_name': 'NAMELSAD'}, inplace=True)
    df['date'] = pd.to_datetime(df['date'])
    grouped = df.groupby('date')
    os.makedirs(output_folder, exist_ok=True)

    colors = ["#00FF00", "#3FBF00", "#7F7F00", "#BF3F00", "#FF0000"]
    cmap = LinearSegmentedColormap.from_list("rate_scale", colors, N=5)
    norm = Normalize(vmin=0, vmax=1)

    for date, group in grouped:
        color_dict = {row['NAMELSAD']: row['color'] for idx, row in group.iterrows()}
        shapefile_gdf['color'] = shapefile_gdf['NAMELSAD'].map(color_dict).fillna('#808080')

        fig, ax = plt.subplots(figsize=(15, 10))  # Increased height to accommodate the colorbar below
        ax.set_xlim([-130, -66])
        ax.set_ylim([24, 50])
        shapefile_gdf.plot(ax=ax, color='lightgrey', edgecolor='black', linewidth=0.4)
        shapefile_gdf.plot(ax=ax, color=shapefile_gdf['color'], edgecolor='black', linewidth=0.4)
        
        # Adjust the main plot's position to leave space for the colorbar
        ax_position = ax.get_position()  # Get the bounding box of the main plot
        ax.set_position([ax_position.x0, ax_position.y0 + ax_position.height * 0.1,  # Adjust bottom
                         ax_position.width, ax_position.height * 0.9])  # Adjust height

        # Create a new axes for the colorbar at the bottom
        colorbar_ax = fig.add_axes([ax_position.x0, ax_position.y0 * 0.1, ax_position.width, 0.03])  # Position for the colorbar
        
        # Create and configure the colorbar
        cbar = ColorbarBase(colorbar_ax, cmap=cmap, norm=norm, orientation='horizontal')
        cbar.set_label('Rate')
        cbar.set_ticks([0.12, 0.37, 0.62, 0.825, 1.0])
        cbar.set_ticklabels(['0.0-0.24', '0.25-0.49', '0.5-0.74', '0.75-0.90', '1.0'])
        
        # Set the title and save the figure
        ax.set_title(title_format.format(date.strftime('%Y-%m-%d')))
        ax.set_axis_off()
        plt.savefig(os.path.join(output_folder, f"{date.strftime('%Y-%m-%d')}.png"), dpi=300, bbox_inches='tight')
        plt.close(fig)

# File paths
csv_file_path = r"C:\Users\A404007\Desktop\Gils Folder\Data Mining\projectdata\ChangedData\aggregated_stats_4BinsMethod14D.csv"
shp_file_path = r'C:\Users\A404007\Desktop\Gils Folder\Data Mining\projectdata\GeoShapeData\tl_2023_us_county\tl_2023_us_county.shp'
output_folder = r'C:\Users\A404007\Desktop\Gils Folder\Data Mining\projectdata\GeoPlot'

# Load shapefile into GeoDataFrame
shape_df = gpd.read_file(shp_file_path)
#shape_df.sort_values(by=['STATEFP', 'NAMELSAD'], ascending=[True, True], inplace=True) #SORTED
# Call the function
title_format = "COVID-19 Incidence on {}"
plot_counties_from_csv(shape_df, csv_file_path, output_folder, title_format)
